In [1]:
from os import environ as env
env['DJANGO_SETTINGS_MODULE'] = 'ov_wag.settings.dev'
env["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
django.setup()
from wagtail.models import Page

ov = Page.objects.get(id=3)
aapb = Page.objects.get(id=59)  

In [2]:
# from home.models import HomePage
# aa = HomePage(title="AAPB")
# aa.save()

In [3]:
from cmless.models import Collection
import json
with open('/home/harpo/gbh/aapb/AAPB2/collections.json') as f:
    data = json.load(f)
    collections = [
        Collection(**item) for item in data
    ]
collections[0].resources

'- [List of Black-Produced TV Shows Nationwide, from 1968-on](https://www.thirteen.org/blog-post/list-of-black-produced-tv-shows-nationwide-from-1968-on/)\n- [Televising Politics in the Black Power Era: Black Journal and Soul!](https://americanarchive.org/exhibits/black-power)\n- [The Soul of Black Identity: Artist Interviews of the Post-Soul Era](https://americanarchive.org/special_collections/soul-of-black-identity)\n- [Basic Black Timeline](https://www.wgbh.org/basic-black-timeline)\n- [Basic Black, "Memorable Episodes"](https://www.pbs.org/video/memorable-episodes-y4aokj/)\n- [50 Years of Basic Black: Then and Now](https://baystatebanner.com/2018/05/16/50-years-of-basic-black-then-and-now/)\n- [GBH News Rooted](https://www.wgbh.org/tv-shows/gbh-news-rooted)'

In [4]:
from aapb_collections.models import AAPBCollection, AAPBRecordsBlock
from cmless.parse import markdownify


In [5]:
# collection = OpenVaultCollection(title='test', content=[('text', 'programatic creation'), ('text', 'with streamfield data!!!'), ('text', markdownify(collections[0].resources))])
# aapb.add_child(instance=collection)
c = collections[0]
markdownify(c.featured)

'<p><a href="/catalog/cpb-aacip_15-99p2w600"><img alt="Black Power on Campus" src="https://s3.amazonaws.com/americanarchive.org/special-collections/cpb-aacip_15-99p2w600.jpg" /></a>\n<a href="/catalog/cpb-aacip_15-9nc5sc3d"><img alt="Equal Rights Amendment" src="https://s3.amazonaws.com/americanarchive.org/special-collections/cpb-aacip_15-9nc5sc3d.jpg" /></a>\n<a href="/catalog/cpb-aacip-15-5370t34j"><img alt="Learning the Hard Way" src="https://s3.amazonaws.com/americanarchive.org/special-collections/hard-way-ss.jpeg" /></a>\n<a href="/catalog/cpb-aacip-15-784j1mm1"><img alt="Surviving Cancer" src="https://s3.amazonaws.com/americanarchive.org/special-collections/basicblack-cancer2.jpeg" /></a>\n<a href="/catalog/cpb-aacip-15-12m64dwb"><img alt="Second Chances" src="https://s3.amazonaws.com/americanarchive.org/special-collections/second-chances-bb.jpeg" /></a>\n<a href="/catalog/cpb-aacip-15-36txc2d4"><img alt="A Conversation with Anita Hill" src="https://s3.amazonaws.com/americanarchi

In [6]:
import re
from typing import List, Dict

def parse_cmless_thumbnail(markdown_string: str) -> List[Dict[str, str]]:
    """
    Parse a markdown string containing cmless images into a list of objects.

    Args:
        markdown_string (str): The markdown string to parse

    Returns:
        List[Dict[str, str]]: List of dictionaries with 'title' and 'image_url' keys
    """
    # Pattern to match ![title](url)
    pattern = r'\!\[([^\]]*)\]\(([^\)]*)\)'
    matches = re.findall(pattern, markdown_string)

    cmless_images = []
    for match in matches:
        title, url = match
        cmless_images.append({
            'title': title.strip(),
            'url': url.strip().split(' ')[0]
        })

    return cmless_images

def parse_featured_markdown(markdown_string: str) -> List[Dict[str, str]]:
    """
    Parse a markdown string containing featured items into a list of objects.
    
    Expected format: [![title](image_url)](link_url)
    
    Args:
        markdown_string (str): The markdown string to parse
        
    Returns:
        List[Dict[str, str]]: List of dictionaries with 'title', 'image_url', and 'link_url' keys
    """
    # Pattern to match [![title](image_url)](link_url)
    pattern = r'\[\!\[([^\]]*)\]\(([^\)]*)\)\]\(([^\)]*)\)'
    
    matches = re.findall(pattern, markdown_string)
    
    featured_items = []
    for match in matches:
        title, image_url, link_url = match
        guid = link_url.split('/')[-1].split('#')[0]  # Extract guid from link_url
        start_time = link_url.split('#at_')[-1] if '#at_' in link_url else None
        featured_items.append({
            'title': title.strip(),
            # 'thumbnail': image_url.strip(),
            'guids': guid.strip(),
            'start_time': start_time.strip().replace('_s', '') if start_time else None
        })
    
    return featured_items


# parse_featured_markdown("[![Ethic for Broadcasting](https://s3.amazonaws.com/americanarchive.org/special-collections/Minow_TVtile.jpg)](/catalog/cpb-aacip_500-kw57jd6j)\n[![The Carnegie Commission](https://s3.amazonaws.com/americanarchive.org/special-collections/Minow_TVtile.jpg)](/catalog/cpb-aacip_507-707wm14c2g)\n[![North Carolina Now](https://s3.amazonaws.com/americanarchive.org/special-collections/Minow_TVtile.jpg)](/catalog/cpb-aacip_129-09j3v33q#at_1173.159414_s)\n[![Public Television Hearings](https://s3.amazonaws.com/americanarchive.org/special-collections/Minow_TVtile.jpg)](/catalog/cpb-aacip_15-07tmpp7v)\n[![New Vistas for Television](https://s3.amazonaws.com/americanarchive.org/special-collections/Minow_TVtile.jpg)](/catalog/cpb-aacip_15-451g1x4f)\n[![Medal of Freedom, NewsHour](https://s3.amazonaws.com/americanarchive.org/special-collections/Minow_TVtile.jpg)](/catalog/cpb-aacip_525-gx44q7rv3h#at_3150.040405_s)")

In [7]:
from wagtail.images.models import Image

def download_image(url: str, title: str | None = None) -> Image | None:
    import requests
    from django.core.files.base import ContentFile
    from wagtail.images import get_image_model

    ImageModel = get_image_model()
    response = requests.get(url)
    if not title:
        title = url.split("/")[-1]
    if response.status_code == 200:
        image = Image(file=ContentFile(response.content, name=title), title=title)
        image.save()
        return image
    else:
        print(f"Failed to download image from {url}")
        return None
# i = Image(title="Example Image", file="Screenshot from 2025-07-19 22-13-28.png")
# i = Image(title="Example Image", file="original_images/WeBelongHere.png")
# https://s3.amazonaws.com/americanarchive.org/special-collections/WeBelongHere.png
# i = download_image("https://s3.amazonaws.com/americanarchive.org/special-collections/WeBelongHere.png")


In [8]:
from aapb_collections.models import AAPBCollection

def create_collection_page(collection):
    content=[
            ('background', markdownify(collection.background)) if collection.background else None,
            ('funders', markdownify(collection.funders)) if collection.funders else None,
            ('help', markdownify(collection.help)) if collection.help else None,
            ('resources', markdownify(collection.resources)) if collection.resources else None,
            ('terms', markdownify(collection.terms)) if collection.terms else None,
            ('timeline', markdownify(collection.timeline)) if collection.timeline else None,
        ]
    content = [item for item in content if item is not None]
    if collection.sort:
        sort = collection.sort.split('+')
        sort_by = sort[0] if len(sort) > 0 else None
        if sort_by == 'asset_date':
            sort_by = 'date'
        elif sort_by == 'asset_title':
            sort_by = 'title'
        sort_order = sort[1] if len(sort) > 1 else None
    if collection.featured:
        featured = parse_featured_markdown(collection.featured)
        featured_items = [('records', record) for record in featured]
    if collection.thumbnail:
        image = parse_cmless_thumbnail(collection.thumbnail)[0]
        thumbnail = download_image(image['url'], title=image['title']) if image else None
    if collection.title.find('<em>') != -1 or collection.title.find('*') != -1:
        display_title = collection.title
        title = collection.title.replace('<em>', '').replace('</em>', '').replace('*', '')
    # return content
    page = AAPBCollection(
        title=title if 'title' in locals() else collection.title,
        display_title=markdownify(display_title) if 'display_title' in locals() else None,
        introduction=markdownify(collection.summary),
        content=content,
        featured_items=featured_items if 'featured_items' in locals() else None,
        sort_by=sort_by if 'sort_by' in locals() else None,
        sort_order=sort_order if 'sort_order' in locals() else None,
        hero_image=thumbnail if 'thumbnail' in locals() else None,
    )
    return page

# create_collection_page(collections[0])

In [9]:
for collection in collections:
    page = create_collection_page(collection)
    print(f"Creating collection: {page}")
    aapb.add_child(instance=page)
    print(f"Added collection: {collection.title}")

/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/elasticsearch/_sync/client/__init__.py:403: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
Exception raised while adding <Image: Say Brother/Basic Black> into the 'default' search backend
Traceback (most recent call last):
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
        (self._dns_host, self.port),
    ...<2 lines>...
        socket_options=self.socket_options,
    )
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
    ~~~~~~~~~~~~^^^^
ConnectionRefusedError: [Errno 111] Connection refuse

Creating collection: Say Brother/Basic Black


Exception raised while adding <AAPBCollection: Say Brother/Basic Black> into the 'default' search backend
Traceback (most recent call last):
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
        (self._dns_host, self.port),
    ...<2 lines>...
        socket_options=self.socket_options,
    )
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
    ~~~~~~~~~~~~^^^^
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/elastic_transport/_node/_http_urllib3.py", line 167, 

Added collection: Say Brother/Basic Black


Exception raised while adding <Image: The Murder of Emmett Till> into the 'default' search backend
Traceback (most recent call last):
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
        (self._dns_host, self.port),
    ...<2 lines>...
        socket_options=self.socket_options,
    )
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
    ~~~~~~~~~~~~^^^^
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/elastic_transport/_node/_http_urllib3.py", line 167, in perf

...

Creating collection: National Federation of Community Broadcasters Programs


Exception raised while adding <AAPBCollection: National Federation of Community Broadcasters Programs> into the 'default' search backend
Traceback (most recent call last):
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
        (self._dns_host, self.port),
    ...<2 lines>...
        socket_options=self.socket_options,
    )
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
    ~~~~~~~~~~~~^^^^
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/elastic_transport/_nod

Added collection: National Federation of Community Broadcasters Programs


Exception raised while adding <Image: Woman> into the 'default' search backend
Traceback (most recent call last):
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
        (self._dns_host, self.port),
    ...<2 lines>...
        socket_options=self.socket_options,
    )
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
    ~~~~~~~~~~~~^^^^
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/harpo/gbh/ov/ov-wag/.venv/lib/python3.13/site-packages/elastic_transport/_node/_http_urllib3.py", line 167, in perform_request
    resp

Creating collection: Woman Series
Added collection: Woman Series
